In [ ]:
# !pip install openai
# !pip install langchain
# !pip install tiktoken
# !pip install faiss-cpu

Langchain concepts to clarify.

[RunnablePassThrough?](https://python.langchain.com/docs/expression_language/how_to/passthrough)

What is a `Runnable` in langchain? 

Its a building block of any composition of components and pieces in langchain that you can actually run.

They are tools for composability.


From the documentation:

```
> A unit of work that can be invoked, batched, streamed, transformed and composed.

> - `invoke/ainvoke`: Transforms a single input into an output.
> - `batch/abatch`: Efficiently transforms multiple inputs into outputs.
> - `stream/astream`: Streams output from a single input as it’s produced.
> - `astream_log`: Streams output and selected intermediate results from an input.

```

![](../assets-resources/attributes_runnable.png)

`RunnableLambda`

In [ ]:
from langchain_core.runnables import RunnableLambda

# Define a custom function
def add_one(x: int) -> int:
    return x + 1

# Create a RunnableLambda and invoke it
runnable = RunnableLambda(add_one)
output = runnable.invoke(5)
print(output)  # Output: 6

[`RunnableSequence`](https://api.python.langchain.com/en/stable/runnables/langchain_core.runnables.base.Runnable.html#langchain_core.runnables.base.Runnable:~:text=runnablesequence%20invokes%20a%20series%20of%20runnables%20sequentially%2C%20with%20one%20runnable%E2%80%99s%20output%20serving%20as%20the%20next%E2%80%99s%20input.%20construct%20using%20the%20%7C%20operator%20or%20by%20passing%20a%20list%20of%20runnables%20to%20runnablesequence.)

RunnableSequence invokes a series of runnables sequentially, with one runnable’s output serving as the next’s input. Construct using the | operator or by passing a list of runnables to RunnableSequence.

In [ ]:
from langchain_core.runnables import RunnableSequence
# Suppose we have a list of numbers
data = [1, 2, 3, 4, 5]

# Preprocess function: Multiply each number by 2
def preprocess(data):
    return [x * 2 for x in data]

# Apply model function: Square each number
def apply_model(data):
    return [x ** 2 for x in data]

# Postprocess function: Sum all the numbers
def postprocess(data):
    return sum(data)

# Create RunnableLambdas
preprocess_runnable = RunnableLambda(preprocess)
apply_model_runnable = RunnableLambda(apply_model)
postprocess_runnable = RunnableLambda(postprocess)

# Create a RunnableSequence and invoke it
sequence = preprocess_runnable | apply_model_runnable | postprocess_runnable
# runnable_sequence = RunnableSequence(sequence)
output = sequence.invoke(data)

print(output)
# or
runnable_sequence = RunnableSequence(first=preprocess_runnable, middle=[apply_model_runnable], last=postprocess_runnable)
runnable_sequence.invoke(data)

[`Runnableparallel`]([`Runnableparallel`](https://api.python.langchain.com/en/stable/runnables/langchain_core.runnables.base.Runnable.html#langchain_core.runnables.base.Runnable:~:text=runnableparallel%20invokes%20runnables%20concurrently%2C%20providing%20the%20same%20input%20to%20each.%20construct%20it%20using%20a%20dict%20literal%20within%20a%20sequence%20or%20by%20passing%20a%20dict%20to%20runnableparallel.)https://api.python.langchain.com/en/stable/runnables/langchain_core.runnables.base.Runnable.html#langchain_core.runnables.base.Runnable:~:text=runnableparallel%20invokes%20runnables%20concurrently%2C%20providing%20the%20same%20input%20to%20each.%20construct%20it%20using%20a%20dict%20literal%20within%20a%20sequence%20or%20by%20passing%20a%20dict%20to%20runnableparallel.)

RunnableParallel invokes runnables concurrently, providing the same input to each. Construct it using a dict literal within a sequence or by passing a dict to RunnableParallel.

In [ ]:
import langchain
print(langchain.__version__)

In [ ]:
from langchain_core.runnables import RunnableLambda

# A RunnableSequence constructed using the `|` operator
sequence = RunnableLambda(lambda x: x + 1) | RunnableLambda(lambda x: x * 2)
sequence.invoke(1) # 4
sequence.batch([1, 2, 3]) # [4, 6, 8]

# A sequence that contains a RunnableParallel constructed using a dict literal
sequence = RunnableLambda(lambda x: x + 1) | {
    'mul_2': RunnableLambda(lambda x: x * 2),
    'mul_5': RunnableLambda(lambda x: x * 5)
}
sequence.invoke(1) # {'mul_2': 4, 'mul_5': 10}


When you build chains, what you're doing is building a type of Runnable!

That could be a RunnableSequence or a RunnableParallel  

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI


llm = ChatOpenAI()
prompt = PromptTemplate.from_template("Summarize this {text}")
prompt2 = PromptTemplate.from_template("Explain this {text}")


chain1 = prompt | llm

chain2 = prompt2 | llm

runnable_parallel = chain1 

In [ ]:
type(chain)

[RunnableParallel can be useful for manipulating the output of one Runnable to match the input format of the next Runnable in a sequence.](https://python.langchain.com/docs/expression_language/how_to/map#:~:text=RunnableParallel%20can%20be,the%0A%E2%80%9Cquestion%E2%80%9D%20key.)

RunnableParallel (aka. RunnableMap) makes it easy to execute multiple Runnables in parallel, and to return the output of these Runnables as a map.

([see it in docs here](https://python.langchain.com/docs/expression_language/how_to/map#:~:text=runnableparallel%20(aka.%20runnablemap)%20makes%20it%20easy%20to%20execute%20multiple%20runnables%20in%20parallel%2C%20and%20to%20return%20the%20output%20of%20these%20runnables%20as%20a%20map.))

In [28]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnableParallel

model = ChatOpenAI()
joke_chain = ChatPromptTemplate.from_template("tell me a joke about {topic}") | model
poem_chain = (
    ChatPromptTemplate.from_template("write a 2-line poem about {topic}") | model
)

map_chain = RunnableParallel(joke=joke_chain, poem=poem_chain)

map_chain.invoke({"topic": "bear"})

{'joke': AIMessage(content="Why don't bears wear shoes? \n\nBecause they already have bear feet!"),
 'poem': AIMessage(content="In forest's embrace, bear roams with might,\nNature's guardian, a majestic sight.")}

RunnableParallel are also useful for running independent processes in parallel, since each Runnable in the map is executed in parallel. For example, we can see our earlier joke_chain, poem_chain and map_chain all have about the same runtime, even though map_chain executes both of the other two.

In [29]:
%%timeit

joke_chain.invoke({"topic": "bear"})

The slowest run took 4.22 times longer than the fastest. This could mean that an intermediate result is being cached.
1.2 s ± 765 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [30]:
%%timeit

poem_chain.invoke({"topic": "bear"})

The slowest run took 9.68 times longer than the fastest. This could mean that an intermediate result is being cached.
1.86 s ± 2.28 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [31]:
%%timeit

map_chain.invoke({"topic": "bear"})

1.29 s ± 684 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


[RunnablePassthrough](https://python.langchain.com/docs/expression_language/how_to/passthrough#:~:text=RunnablePassthrough%20allows%20to,pass%20it%20through.)

Allows you to pass inputs unchanged or with addition of new keys.

Usually you would connect this with `RunnableParallel` to assign data to new key in the map.

If you just call it, it will take the input and pass it along.

In [32]:
from langchain_core.runnables import RunnablePassthrough

# Create a RunnablePassthrough
passthrough = RunnablePassthrough()
passthrough.invoke(5) # 5

5

But if you call it with the `.assign(...)` mthod, then it will take the input, and add extra arguments passed to the assign function. 

The key being added has to be a lambda function.

In [51]:
from langchain.schema.runnable import RunnableParallel, RunnablePassthrough

# Passthrough with assignment, adds an extra key-value pair
runnable = RunnablePassthrough.assign(extra_key_add_5=lambda x:  x["num"]+5)
input_data = {"num": 1}
result = runnable.invoke(input_data)
# Output: {'num': 1, 'extra_value': 42}
result

{'num': 1, 'extra_key_add_5': 6}

In combination with RunnableParallel:

In [52]:
runnable = RunnableParallel(
    origin=RunnablePassthrough(),
    modified=lambda x: x+1
)

runnable.invoke(1) # {'origin': 1, 'modified': 2}

{'origin': 1, 'modified': 2}

In [63]:
# modified example from langchain docs: https://api.python.langchain.com/en/stable/runnables/langchain_core.runnables.passthrough.RunnablePassthrough.html?highlight=runnablepassthrough#langchain_core.runnables.passthrough.RunnablePassthrough 
# or for an "LLM" example:

def fake_llm(prompt: str) -> str: # Fake LLM for the example
    return "completion"

chain = RunnableLambda(fake_llm) | {
    'original': RunnablePassthrough(), # Original LLM output
    'parsed': lambda text: text[::-1] + " (applied the parsing logic)" # Parsing logic
}

chain.invoke('hello') # {'original': 'completion', 'parsed': 'noitelpmoc'}

{'original': 'completion', 'parsed': 'noitelpmoc (applied the parsing logic)'}

Ok so we have these 4 types of main objects:

- `RunnableLambda`
- `RunnableSequence`
- `RunnablePassthrough`
- `RunnableParallel`

combined they make up kind of the core building block system of langchain allow you to create complex inner logics powered by llms. 

Let's create some fun examples below:

In [71]:
from langchain.chat_models import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough

def fun_llm(prompt: str):
    return ChatOpenAI().predict(f"Make this funny: {prompt}")


def corporate_llm(prompt: str):
    return ChatOpenAI().predict(f"Make this topic a subject of a very corporate email: {prompt}")

fun_chain = RunnableLambda(fun_llm) | {
    "original-text-input": RunnablePassthrough(), 
    "make-it-corporate": lambda x: corporate_llm(x)
}


fun_chain.invoke("gorillas")

{'original-text-input': 'Why did the gorilla bring a ladder to the bar?\n\nBecause he heard the drinks were on the house!',
 'make-it-corporate': 'Subject: Humorous Anecdote - Gorilla\'s Ingenious Solution to Access the Bar\n\nDear [Colleagues/Team Members],\n\nI hope this email finds you well. Today, I wanted to lighten the mood and share a lighthearted anecdote that recently made its way to my inbox. We all deserve a good laugh amidst our busy corporate lives, don\'t we?\n\nThe story revolves around a gorilla who, in an unexpected turn of events, decided to visit a local bar. Now, you might be wondering why this seemingly ordinary occurrence has found its way into our corporate correspondence. Well, this tale goes beyond a mere joke; it showcases the power of ingenuity and problem-solving skills even in the most unexpected situations.\n\nSo, without further ado, let\'s delve into the story. Our gorilla friend, upon hearing that the drinks were "on the house," embarked upon a mission 

In [76]:
from langchain_core.runnables import RunnablePassthrough, RunnableParallel

def fake_llm(prompt: str) -> str: # Fake LLM for the example
    return "completion"

runnable = {
    'llm1':  fake_llm,
    'llm2':  fake_llm,
} | RunnablePassthrough.assign(
    total_chars=lambda inputs: len(inputs['llm1'] + inputs['llm2'])
  )

runnable.invoke('hello')
# {'llm1': 'completion', 'llm2': 'completion', 'total_chars': 20}

langchain_core.runnables.base.RunnableSequence

Breaking down whats happening:

We create a `runnable` object, which is an instance of the `RunableSequence` object. This chain takes in as input a simple string: `hello`, and what it does is it 
simulates passing that same string to 2 `different` llms (which in this case are the same) and applies the logic of each of these llms as well as a third logic described by the new extra key added by the RunnablePassthrough.assign() method, that takes as input the output of the `llm1` and `llm2` as input to combine it through the lambda function associated with the new extra key created: `total_chars`. 

10